In [ ]:
import os
import cv2
import textwrap
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from transformers import CLIPProcessor, CLIPModel
from diffusers import StableDiffusionPipeline





In [ ]:
def generate_manga_panel(prompt, style="shonen", expression="neutral", output_path="panel.png"):
    model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    full_prompt = f"{prompt}, manga-style, {style}, character expression: {expression}, black and white"
    image = model(full_prompt).images[0]
    image.save(output_path)
    print(f"Manga panel saved as {output_path}")
    return output_path

In [ ]:
def detect_and_remember_character(image_path, database):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    embedding = model.get_image_features(**inputs)
    database[image_path] = embedding.detach().numpy()
    print(f"Character detected and stored for {image_path}")



In [ ]:
def add_speech_bubble(image_path, text, position):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("arial.ttf", 18)
    except:
        font = ImageFont.load_default()
    max_chars = 22
    if len(text) > max_chars:
        text = text[:max_chars] + "..."
    x, y = position
    bubble_width = 160
    bubble_height = 80
    draw.ellipse([x, y, x + bubble_width, y + bubble_height], fill="white", outline="black", width=3)
    draw.text((x + 15, y + 30), text, fill="black", font=font)
    image.save(image_path)
    print(f"Speech bubble added to {image_path}")



In [ ]:
def create_manga_collage(panel_paths, output_path="manga_page.png", panels_per_row=2, margin=20):
    images = [Image.open(p) for p in panel_paths]
    max_width = max(img.width for img in images)
    max_height = max(img.height for img in images)
    resized = [img.resize((max_width, max_height)) for img in images]
    rows = (len(images) + panels_per_row - 1) // panels_per_row
    total_width = panels_per_row * max_width + (panels_per_row + 1) * margin
    total_height = rows * max_height + (rows + 1) * margin
    collage = Image.new("RGB", (total_width, total_height), "white")
    for idx, img in enumerate(resized):
        row = idx // panels_per_row
        col = idx % panels_per_row
        x = margin + col * (max_width + margin)
        y = margin + row * (max_height + margin)
        collage.paste(img, (x, y))
    collage.save(output_path)
    print(f"Manga page saved as {output_path}")



In [ ]:
def create_fullpage_manga_collage(panel_paths, output_path="manga_page.png"):
    page_width, page_height = 2480, 3508
    background = Image.new("RGB", (page_width, page_height), "white")
    draw = ImageDraw.Draw(background)
    panels = [Image.open(p) for p in panel_paths]
    if not panels:
        print("No panels provided.")
        return
    import random
    num_rows = random.randint(2, 4)
    panel_grid = [[] for _ in range(num_rows)]
    for i, panel in enumerate(panels):
        panel_grid[i % num_rows].append(panel)
    current_y = 0
    for row_panels in panel_grid:
        row_height = page_height // num_rows
        num_cols = len(row_panels)
        current_x = 0
        for panel in row_panels:
            col_width = page_width // num_cols
            resized = panel.resize((col_width, row_height))
            background.paste(resized, (current_x, current_y))
            draw.rectangle([current_x, current_y, current_x + col_width, current_y + row_height], outline="black", width=5)
            current_x += col_width
        current_y += row_height
    background.save(output_path)
    print(f"Full-page manga saved as {output_path}")



In [ ]:

character_db = {}

panel1 = generate_manga_panel(
    "A full moon glows over a war-torn battlefield littered with swords and fallen warriors. Two samurai face each other from a distance.",
    "seinen", "tense", "panel1.png"
)

panel2 = generate_manga_panel(
    "Close-up of a samurai drawing his katana slowly, eyes full of sorrow and rage. The wind blows his hair and clothes.",
    "shonen", "serious", "panel2.png"
)

panel3 = generate_manga_panel(
    "The rival samurai with a confident smirk, holding his katana on his shoulder under the crimson moonlight.",
    "seinen", "smug", "panel3.png"
)

panel4 = generate_manga_panel(
    "Dynamic clash of both samurai under the moonlight, ink splashes mimicking blood and impact.",
    "shonen", "intense", "panel4.png"
)

panel5 = generate_manga_panel(
    "Protagonist samurai wounded, blood dripping, still standing firm with burning eyes.",
    "seinen", "gritty", "panel5.png"
)

panel6 = generate_manga_panel(
    "Final blow as one samurai strikes while the moon shines bright, thick diagonal ink lines.",
    "shonen", "final", "panel6.png"
)

for panel in [panel1, panel2, panel3, panel4, panel5, panel6]:
    detect_and_remember_character(panel, character_db)

add_speech_bubble(panel1, "Tonight... it ends.", (150, 60))
add_speech_bubble(panel2, "I swore on his grave I would finish this!", (100, 200))
add_speech_bubble(panel3, "You’re still chasing ghosts, brother?", (180, 150))
add_speech_bubble(panel4, "HAAAARGH!", (200, 100))
add_speech_bubble(panel5, "I won’t fall... not yet...", (160, 220))
add_speech_bubble(panel6, "Forgive me.", (180, 100))

create_manga_collage([panel1, panel2, panel3, panel4, panel5, panel6], "final_manga.png", panels_per_row=2, margin=30)
create_fullpage_manga_collage([panel1, panel2, panel3, panel4, panel5])
